In [4]:
import cv2
import numpy as np

# Carica le immagini
img_ref = cv2.imread('output2/template.png', cv2.IMREAD_GRAYSCALE)  # Immagine di riferimento
img_acquired = cv2.imread('output2/106816.png', cv2.IMREAD_GRAYSCALE)  # Immagine acquisita

# Inizializza il rilevatore ORB
orb = cv2.ORB_create()

# Trova i keypoints e i descrittori nelle immagini
kp_ref, des_ref = orb.detectAndCompute(img_ref, None)
kp_acquired, des_acquired = orb.detectAndCompute(img_acquired, None)

# Inizializza il matcher di descrittori
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Esegui il matching dei descrittori tra le due immagini
matches = bf.match(des_ref, des_acquired)

# Ordina i match per distanza
matches = sorted(matches, key = lambda x:x.distance)

# Disegna i primi 50 match
img_matches = cv2.drawMatches(img_ref, kp_ref, img_acquired, kp_acquired, matches[:50], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Mostra l'immagine risultante
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Imposta una soglia per determinare se le immagini sono simili
if len(matches) > 30:  # Se ci sono più di 30 match, considera che l'oggetto è presente
    print("L'oggetto è presente nell'immagine acquisita.")
else:
    print("L'oggetto non è presente nell'immagine acquisita.")


L'oggetto non è presente nell'immagine acquisita.


In [2]:
import cv2
import numpy as np

# Carica l'immagine di template e l'immagine acquisita
img_template = cv2.imread('template_image.jpg', cv2.IMREAD_GRAYSCALE)  # Immagine di riferimento (template)
img_acquired = cv2.imread('acquired_image.jpg', cv2.IMREAD_GRAYSCALE)  # Immagine acquisita

# Esegui il Template Matching
result = cv2.matchTemplate(img_acquired, img_template, cv2.TM_CCOEFF_NORMED)

# Trova la posizione con la massima corrispondenza
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# Imposta una soglia di corrispondenza (ad esempio, 0.8 indica una corrispondenza alta)
threshold = 0.8

# Se la corrispondenza è sopra la soglia, considera che l'oggetto è presente
if max_val >= threshold:
    print("L'oggetto è presente nell'immagine acquisita.")
    
    # Disegna un rettangolo per visualizzare dove è stato trovato il template
    h, w = img_template.shape
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(img_acquired, top_left, bottom_right, (0, 255, 0), 2)
    
    # Mostra l'immagine con il rettangolo disegnato
    cv2.imshow('Detected Template', img_acquired)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("L'oggetto non è presente nell'immagine acquisita.")


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\templmatch.cpp:588: error: (-215:Assertion failed) corr.rows <= img.rows + templ.rows - 1 && corr.cols <= img.cols + templ.cols - 1 in function 'cv::crossCorr'


In [1]:
import cv2
import numpy as np

# Percorsi dei file caricati
template_path = "output2/template.png"
#target_path = "output2/106713.png"
target_path = "output4/77123.png"

# Caricare le immagini
template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)

# Eseguire il template matching
result = cv2.matchTemplate(target, template, cv2.TM_CCOEFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

# Ottenere le coordinate del riquadro di corrispondenza
template_height, template_width = template.shape
top_left = max_loc
bottom_right = (top_left[0] + template_width, top_left[1] + template_height)

# Disegnare il riquadro di corrispondenza sull'immagine target
target_marked = cv2.cvtColor(target, cv2.COLOR_GRAY2BGR)
cv2.rectangle(target_marked, top_left, bottom_right, (0, 255, 0), 2)

# Salvare l'immagine risultante con il riquadro disegnato
result_path = "result_marked.png"
cv2.imwrite(result_path, target_marked)

# Restituire il valore massimo di correlazione e il percorso del risultato
max_val, result_path


(0.4909091889858246, 'result_marked.png')

In [11]:
import cv2
import numpy as np

# Percorsi dei file caricati
template_path = "output2/template.png"
target_path = "output4/76780.png"

# Caricare le immagini
template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)

# Ottenere le dimensioni del template
template_height, template_width = template.shape

# Parametro di scala per il multiscaling
scales = np.linspace(0.5, 1.5, 10)[::-1]  # Scala da 0.5 a 1.5 in 10 passaggi (invertito per ridurre prima)

# Inizializzare il valore massimo di correlazione
max_val = -1
max_loc = None
best_scale = 1

# Eseguire il template matching su diversi livelli di scala
for scale in scales:
    # Ridimensionare l'immagine target
    target_resized = cv2.resize(target, (0, 0), fx=scale, fy=scale)

    # Controllare se l'immagine ridimensionata è abbastanza grande da contenere il template
    if target_resized.shape[0] < template_height or target_resized.shape[1] < template_width:
        continue

    # Eseguire il template matching
    result = cv2.matchTemplate(target_resized, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val_current, min_loc, max_loc_current = cv2.minMaxLoc(result)

    # Aggiornare il miglior valore di correlazione e la posizione
    if max_val_current > max_val:
        max_val = max_val_current
        max_loc = max_loc_current
        best_scale = scale

# Calcolare le dimensioni del template trovato in base alla scala
template_height_scaled = int(template_height * best_scale)
template_width_scaled = int(template_width * best_scale)

# Calcolare la posizione del riquadro di corrispondenza nel sistema di coordinate originali
top_left = (int(max_loc[0] / best_scale), int(max_loc[1] / best_scale))
bottom_right = (int((max_loc[0] + template_width_scaled) / best_scale),
                int((max_loc[1] + template_height_scaled) / best_scale))

# Ridimensionare l'immagine target alla scala corretta
target_resized_final = cv2.resize(target, (0, 0), fx=best_scale, fy=best_scale)

# Disegnare il riquadro di corrispondenza sull'immagine target
target_marked = cv2.cvtColor(target_resized_final, cv2.COLOR_GRAY2BGR)
cv2.rectangle(target_marked, top_left, bottom_right, (0, 255, 0), 2)

# Salvare l'immagine risultante con il riquadro disegnato
result_path = "result_marked.png"
cv2.imwrite(result_path, target_marked)

# Restituire il valore massimo di correlazione e il percorso del risultato
print(f"Max correlation value: {max_val}")
print(f"Result saved at: {result_path}")


Max correlation value: 0.6649532914161682
Result saved at: result_marked.png


In [16]:
import cv2
import numpy as np

# Percorsi dei file caricati
template_path = "output2/template2.png"
target_path = "output2/106622.png"

# Caricare le immagini
template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
target = cv2.imread(target_path, cv2.IMREAD_GRAYSCALE)

# Ottenere le dimensioni del template
template_height, template_width = template.shape

# Parametro di scala per il multiscaling
scales = np.linspace(0.5, 1.5, 10)[::-1]  # Scala da 0.5 a 1.5 in 10 passaggi (invertito per ridurre prima)

# Inizializzare il valore massimo di correlazione
max_val = -1
max_loc = None
best_scale = 1

# Eseguire il template matching su diversi livelli di scala
for scale in scales:
    # Ridimensionare l'immagine target
    target_resized = cv2.resize(target, (0, 0), fx=scale, fy=scale)

    # Controllare se l'immagine ridimensionata è abbastanza grande da contenere il template
    if target_resized.shape[0] < template_height or target_resized.shape[1] < template_width:
        continue

    # Eseguire il template matching
    result = cv2.matchTemplate(target_resized, template, cv2.TM_CCOEFF_NORMED)
    min_val, max_val_current, min_loc, max_loc_current = cv2.minMaxLoc(result)

    # Aggiornare il miglior valore di correlazione e la posizione
    if max_val_current > max_val:
        max_val = max_val_current
        max_loc = max_loc_current
        best_scale = scale

# Calcolare le dimensioni del template trovato in base alla scala
template_height_scaled = int(template_height * best_scale)
template_width_scaled = int(template_width * best_scale)

# Calcolare la posizione del riquadro di corrispondenza nel sistema di coordinate originali
top_left = (int(max_loc[0] / best_scale), int(max_loc[1] / best_scale))
bottom_right = (int((max_loc[0] + template_width_scaled) / best_scale),
                int((max_loc[1] + template_height_scaled) / best_scale))

# Ridimensionare l'immagine target finale (per mostrare la scala corretta)
target_resized_final = cv2.resize(target, (0, 0), fx=best_scale, fy=best_scale)

# Disegnare il riquadro di corrispondenza sull'immagine target
target_marked = cv2.cvtColor(target_resized_final, cv2.COLOR_GRAY2BGR)
cv2.rectangle(target_marked, top_left, bottom_right, (0, 255, 0), 2)

# Aggiungere il testo con il valore massimo di correlazione
cv2.putText(target_marked, f'Max Corr: {max_val:.4f}', (top_left[0], top_left[1] - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

# Salvare l'immagine risultante con il riquadro disegnato
result_path = "result_marked.png"
cv2.imwrite(result_path, target_marked)

# Restituire il valore massimo di correlazione e il percorso del risultato
print(f"Max correlation value: {max_val}")
print(f"Result saved at: {result_path}")


Max correlation value: 0.744809627532959
Result saved at: result_marked.png
